In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install  \
    "langchain>=0.0.350" \
    "transformers>=4.24,<5" \
    sqlalchemy -U \
    "faiss-cpu>=1.7,<2" \
    "pypdf>=3.8,<4" \
    pinecone-client==2.2.4 \
    apache-beam==2.52. \
    tiktoken==0.5.2 \
    "ipywidgets>=7,<8" \
    matplotlib==3.8.2 \
    anthropic==0.9.0

%pip install datasets==2.15.0

%pip install numexpr==2.8.8

%pip install --quiet \
    xmltodict==0.13.0  \
    duckduckgo-search  \
    yfinance  \
    pandas_datareader  \
    langchain_experimental \
    pysqlite3 \
    google-search-results

%pip install --quiet beautifulsoup4==4.12.2

%pip install -qU "faiss-cpu>=1.7,<2" \
                      "langchain>=0.0.350" \
                      "pypdf>=3.8,<4" \
                      "ipywidgets>=7,<8"
%pip install python-dotenv

In [ ]:
%pip install -r requirements.txt

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [3]:
import json
import os
import sys
import boto3
import botocore
from os.path import join, dirname
from dotenv import load_dotenv
from utils import bedrock, print_ww

load_dotenv()

config = {
    os.environ["AWS_DEFAULT_REGION"],
    os.environ.get("AWS_PROFILE", None),
    os.environ.get("BEDROCK_ASSUME_ROLE", None)
}

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=True
)

Create new client
  Using region: us-west-2
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


In [4]:
prompt_data = """Command: Write me a blog about making strong business decisions as a leader.

Blog:
"""

In [5]:
try:

    body = json.dumps({"inputText": prompt_data})
    modelId = "amazon.titan-tg1-large"
    accept = "application/json"
    contentType = "application/json"

    response = boto3_bedrock.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body.get("results")[0].get("outputText"))

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error

You don't have access to the model with the specified model ID.                
To troubeshoot this issue please refer to the following resources.                 
https://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html                 
https://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html



In [7]:
# Calling with context using anthropic

from langchain.llms.bedrock import Bedrock

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.5,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

textgen_llm = Bedrock(model_id = "anthropic.claude-v2",
                    client = boto3_bedrock, 
                    model_kwargs = inference_modifier 
                    )

from langchain.prompts import PromptTemplate

# Create a prompt template that has multiple input variables
multi_var_prompt = PromptTemplate(
    input_variables=["customerServiceManager", "customerName", "feedbackFromCustomer"], 
    template="""

Human: Create an apology email from the Service Manager {customerServiceManager} to {customerName} in response to the following feedback that was received from the customer: 
<customer_feedback>
{feedbackFromCustomer}
</customer_feedback>

Assistant:"""
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(customerServiceManager="Bob", 
                                 customerName="John Doe", 
                                 feedbackFromCustomer="""Hello Bob,
     I am very disappointed with the recent experience I had when I called your customer support.
     I was expecting an immediate call back but it took three days for us to get a call back.
     The first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.
     We are very unhappy with the response provided and may consider taking our business elsewhere.
     """
     )

num_tokens = textgen_llm.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

response = textgen_llm(prompt)

email = response[response.index('\n')+1:]

print_ww(email)

Our prompt has 127 tokens

I want to sincerely apologize for the poor service experience you recently had with our customer
support team. Receiving your feedback was disappointing, but I appreciate you taking the time to
share your experience candidly.

We strive to provide excellent customer service and quick issue resolution. I am very sorry that we
failed to meet your expectations in this instance. A three day delay for a callback and incorrect
initial troubleshooting suggestions are unacceptable. I completely understand your frustration.

I have discussed your experience in detail with our support team to identify where we went wrong. We
will be implementing additional training and adjusting our call back procedures to ensure this does
not happen again.

I appreciate the opportunity you have given us to earn back your trust. Please let me know if there
is anything else we can do to make this right. Your business is important to us and we are committed
to providing you the excellent